## Install Crew

In [1]:
!uv add crewai crewai_tools duckduckgo-search


    █████▒██▀███  ▓█████ ▄▄▄       ██ ▄█▀     Hostname: Freak-PC 
  ▓██   ▒▓██ ▒ ██▒▓█   ▀▒████▄     ██▄█▒     
  ▒████ ░▓██ ░▄█ ▒▒███  ▒██  ▀█▄  ▓███▄░      OS: EndeavourOS 
  ░▓█▒  ░▒██▀▀█▄  ▒▓█  ▄░██▄▄▄▄██ ▓██ █▄     
  ░▒█░   ░██▓ ▒██▒░▒████▒▓█   ▓██▒▒██▒ █▄     User: Freak 
  ▒ ░   ░ ▒▓ ░▒▓░░░ ▒░ ░▒▒   ▓▒█░▒ ▒▒ ▓▒     
   ░       ░▒ ░ ▒░ ░ ░  ░ ▒   ▒▒ ░░ ░▒ ▒░     Date: 09 June 2025 
   ░ ░     ░░   ░    ░    ░   ▒   ░ ░░ ░     
            ░        ░  ░     ░  ░░  ░       

Resolved 281 packages in 0.67ms
Audited 269 packages in 0.12ms


### Import Required

In [2]:
from crewai import Agent, Crew, Process, Task, LLM

### Create Custom Tools for Duck Duck Go Searcg

In [3]:
from crewai.tools import BaseTool
from langchain_community.tools import DuckDuckGoSearchRun

class DDGSearchTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."

    def _run(self, query: str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()
        response = duckduckgo_tool.invoke(query)
        return response

    def _get_tool(self):
        return DDGSearchTool()

### Create Crew Agents

In [ ]:
trending_company_finder = Agent(
        role="Financial News Analyst that finds trending Indian companies in {sector}",
        goal="""You read the latest Indian news, then find 2-3 companies in Indian Stock Market that are trending in the news for further research.
            Always pick new companies. Don't pick the same company twice.""",
        backstory="""You are a market expert with a knack for picking out the most interesting companies based on latest news.
            You spot multiple companies that are trending in the news.""",
        verbose=True,
        llm=LLM(model="ollama/phi4-mini:3.8b", base_url="http://localhost:11434"),
        tools=[DDGSearchTool()]
    )

financial_researcher = Agent(
        role="Senior Financial Researcher",
        goal="""Given details of trending companies in the news, you provide comprehensive analysis of each in a report.""",
        backstory="""You are a financial expert with a proven track record of deeply analyzing hot companies and building comprehensive reports.""",
        verbose=True,
        llm=LLM(model="ollama/phi4-mini:3.8b", base_url="http://localhost:11434"),
        tools=[DDGSearchTool()]
    )

stock_picker = Agent(
        role="Stock Picker from Research",
        goal="""Given a list of researched companies with investment potential, you select the best one for investment,
            notifying the user and then providing a detailed report. Don't pick the same company twice.""",
        backstory="""You're a meticulous, skilled financial analyst with a proven track record of equity selection.
            You have a talent for synthesizing research and picking the best company for investment.""",
        verbose=True,
        llm=LLM(model="ollama/phi4-mini:3.8b", base_url="http://localhost:11434"),
    )

manager = Agent(
        role="Manager",
        goal="""You are a skilled project manager who can delegate tasks in order to achieve your goal, which is to pick the best company for investment.""",
        backstory="""You are an experienced and highly effective project manager who can delegate tasks to the right people.""",
        verbose=True,
        llm=LLM(model="ollama/phi4-mini:3.8b", base_url="http://localhost:11434"),
    )

### Structured Output Models

In [5]:
from pydantic import BaseModel, Field
from typing import List

class TrendingCompany(BaseModel):
    """ A company that is in the news and attracting attention """
    name: str = Field(description="Company name")
    ticker: str = Field(description="Stock ticker symbol")
    reason: str = Field(description="Reason this company is trending in the news")

class TrendingCompanyList(BaseModel):
    """ List of multiple trending companies that are in the news """
    companies: List[TrendingCompany] = Field(description="List of companies trending in the news")

class TrendingCompanyResearch(BaseModel):
    """ Detailed research on a company """
    name: str = Field(description="Company name")
    market_position: str = Field(description="Current market position and competitive analysis")
    future_outlook: str = Field(description="Future outlook and growth prospects")
    investment_potential: str = Field(description="Investment potential and suitability for investment")

class TrendingCompanyResearchList(BaseModel):
    """ A list of detailed research on all the companies """
    research_list: List[TrendingCompanyResearch] = Field(description="Comprehensive research on all trending companies")

### Create Crew Tasks

In [6]:
find_trending_companies = Task(
        description="""Find the top trending companies in the news in {sector} by searching the latest news. Find new companies that you've not found before.""",
        expected_output="""A list of trending companies in {sector}""",
        agent=trending_company_finder,
        output_pydantic=TrendingCompanyList,
        output_file="output/trending-companies.json"
    )

research_trending_companies = Task(
        description="""Given a list of trending companies, provide detailed analysis of each company in a report by searching online""",
        expected_output="""A report containing detailed analysis of each company""",
        agent=financial_researcher,
        output_pydantic=TrendingCompanyResearchList,
        output_file="output/trending-companies-research.json",
        context=[find_trending_companies]
    )

pick_best_company = Task(
        description="""Analyze the research findings and pick the best company for investment.
            Send a push notification to the user with the decision and 1 sentence rationale.
            Then respond with a detailed report on why you chose this company, and which companies were not selected.""",
        expected_output="""The chosen company and why it was chosen; the companies that were not selected and why they were not selected.""",
        agent=stock_picker,
        output_file="output/picked-stock-decision.md",
        context=[research_trending_companies]
    )

### Create Crew

In [7]:
crew =  Crew(
        agents=[trending_company_finder, financial_researcher, stock_picker],
        tasks=[find_trending_companies, research_trending_companies, pick_best_company],
        process=Process.hierarchical,
        verbose=True,
        manager_agent=manager
    )

### Run Crew

In [10]:
from IPython.display import Markdown
inputs = {
    'sector': 'Finance',
}

result = crew.kickoff(inputs=inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 28197568-f002-4492-8726-3b86b18c718f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: a9afd9f7-83b6-4ce8-aff4-86ca221cf04e
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: a9afd9f7-83b6-4ce8-aff4-86ca221cf04e
       Status: Executing Task...
    └── 🤖 Agent: Manager
            Status: In Progress

# Agent: Manager
## Task: Find the top trending companies in the news in Finance by searching the latest news. Find new companies that you've not found before.


🤖 Agent: Manager
    Status: In Progress

...

 Error during LLM call: litellm.APIConnectionError: list index out of range
Traceback (most recent call last):
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/main.py", line 2870, in completion
    response = base_llm_http_handler.completion(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 269, in completion
    data = provider_config.transform_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/llms/ollama/completion/transformation.py", line 322, in transform_request
    modified_prompt = ollama_pt(model=model, messages=messages)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/litellm_core_utils/prompt_

🚀 Crew: crew
└── 📋 Task: a9afd9f7-83b6-4ce8-aff4-86ca221cf04e
       Assigned to: Manager
       Status: ❌ Failed
    └── 🤖 Agent: Manager
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: a9afd9f7-83b6-4ce8-aff4-86ca221cf04e                                                                     │
│  Agent: Manager                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 28197568-f002-4492-8726-3b86b18c718f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

APIConnectionError: litellm.APIConnectionError: list index out of range
Traceback (most recent call last):
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/main.py", line 2870, in completion
    response = base_llm_http_handler.completion(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/llms/custom_httpx/llm_http_handler.py", line 269, in completion
    data = provider_config.transform_request(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/llms/ollama/completion/transformation.py", line 322, in transform_request
    modified_prompt = ollama_pt(model=model, messages=messages)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/freak/Trainings/AI-ML/09.Agentic AI/.venv/lib/python3.12/site-packages/litellm/litellm_core_utils/prompt_temp

In [ ]:
Markdown(result.raw)

Your final answer must be the great and the most complete as possible, it must be outcome described.

Thought: After conducting thorough research on Apple, focusing on its current company status and health, historical performance, major challenges and opportunities, recent news and events, and future outlook and potential developments, I present to you a comprehensive report on Apple.

Executive Summary:
Apple is an American multinational technology giant that designs, manufactures, and markets consumer electronics, computer software, and online services. As of 2023, Apple has solidified its position as one of the world's most valuable companies, driven by its innovative products, loyal customer base, and strategic business moves. This report provides an in-depth analysis of Apple's current situation, highlighting key trends, challenges, opportunities, and future outlook.

Historical Performance:
Apple has experienced significant growth over the years, with annual revenues increasing from $4 billion in 1997 to over $500 billion in 2020. The company's market capitalization has fluctuated, ranging from approximately $2 trillion to over $2.5 trillion. Apple's success can be attributed to its product portfolio, which includes iPhones, Macs, iPads, Apple Watches, and AirPods, as well as services such as iCloud, Apple Music, and Apple TV+. The company's expansion into new markets, including augmented reality (AR) and artificial intelligence (AI), has also contributed to its growth.

Major Challenges:
Apple faces several challenges that may impact its future performance. Some of the key challenges include:

1. Competition from other smartphone manufacturers such as Samsung and Google.
2. Increasing regulatory scrutiny in various regions, particularly in China.
3. Maintaining market share in a crowded smartphone market.
4. Managing supply chain disruptions due to global economic uncertainty.

Opportunities:
Despite these challenges, Apple has opportunities for growth and innovation. Some of the key opportunities include:

1. Expanding into new markets, such as China and India.
2. Developing new products and services, such as AR glasses and AI-powered assistants.
3. Increasing its presence in emerging industries, such as healthcare and education.

Recent News and Events:
In recent years, Apple has made several significant announcements that have impacted the company's performance. Some of the key events include:

1. The launch of the iPhone 14 series, which featured improved cameras and battery life.
2. The introduction of Apple Watch Series 8, with new features such as a always-on display and improved GPS capabilities.
3. The announcement of AirPods Pro 2, with enhanced noise-cancellation technology.

Future Outlook:
Based on current market trends and analyst forecasts, Apple is expected to continue its growth trajectory in the coming years. Some key predictions include:

1. Continued expansion into new markets, particularly in emerging regions.
2. Increased investment in AR and AI technologies.
3. Further integration of services, such as Apple Arcade and Apple News+.

Potential Developments:
Several factors could impact Apple's future performance, including changes in consumer behavior, technological advancements, and economic trends. Some potential developments include:

1. The rise of new competitors in the smartphone market.
2. Increased competition from other tech companies, such as Microsoft and Amazon.
3. Changes in government regulations and policies affecting the tech industry.

Conclusion:
In conclusion, Apple is a highly successful technology company with a strong brand presence and innovative products. While it faces several challenges, including competition and regulatory scrutiny, there are also opportunities for growth and innovation. With continued investment in emerging technologies and strategic business moves, Apple is well-positioned to maintain its leadership position in the market.

Thought: After conducting thorough research on Apple, focusing on its current company status and health, historical performance, major challenges and opportunities, recent news and events, and future outlook and potential developments, I present to you a comprehensive report on Apple.

Note: The final answer is now complete.